In [1]:
#Importing Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,10)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

In [2]:
#Reading the data set

dff=pd.read_csv("Bengaluru_House_Data.csv")

In [3]:
dff

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


### step1 - Problem and Data Identification

In [4]:
dff.shape

(13320, 9)

In [5]:
dff.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

In [6]:
#so it has 13320 rows and 9 column

In [7]:
# Now lets check how much numerical  column inside the data set

In [8]:
numc=dff.select_dtypes(np.number).columns

In [9]:
numc

Index(['bath', 'balcony', 'price'], dtype='object')

In [10]:
numc.shape

(3,)

In [11]:
#Next i just assuming 'area type','balcony','society','availability', is not important to the dataset
#you can either drop or keep . u can either check the correlation and drop it. for making it minimal i just droping it 

In [12]:
dff = dff.drop(['area_type','society','balcony','availability'],axis=1)
dff.shape

(13320, 5)

In [13]:
dff

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00
...,...,...,...,...,...
13315,Whitefield,5 Bedroom,3453,4.0,231.00
13316,Richards Town,4 BHK,3600,5.0,400.00
13317,Raja Rajeshwari Nagar,2 BHK,1141,2.0,60.00
13318,Padmanabhanagar,4 BHK,4689,4.0,488.00


### step2 - EDA,Feature Engineering and Feature Selection 

In [14]:
# checking columns has null or not

In [15]:
dff.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [16]:
dff['bath'].median()

2.0

In [17]:
# so in here we can see that location has null count 1, size has a null count of 16 and bath has a null count 73

**Now i am going to handle this null value using Center Tendency Method**

In [18]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13319 non-null  object 
 1   size        13304 non-null  object 
 2   total_sqft  13320 non-null  object 
 3   bath        13247 non-null  float64
 4   price       13320 non-null  float64
dtypes: float64(2), object(3)
memory usage: 520.4+ KB


In [19]:
dfz=dff['size']
dff.drop(['size'],axis=1,inplace=True)

**reason i store size in column, cuz its value is combination of number and object 
so if i use mode in size the mepty space is filled with whitefield value ,it leads to error 
in converting size to int format that y i spearate store it . and i i will slice number from it
.then i convert to int**

In [20]:
# in here we fill mode of column if datatype of the column is object
#if dtype of column is numerical  fill it with median

In [21]:
for i in dff:
    if (dff[i].dtypes=='object'):
        dff.fillna(dff[i].mode()[0],inplace=True)
    else:
        dff.fillna(dff[i].median(),inplace=True)

In [22]:
dff.isnull().sum()

location      0
total_sqft    0
bath          0
price         0
dtype: int64

In [23]:
dfz.mode()

0    2 BHK
dtype: object

In [24]:
dfz.fillna('2 BHK',inplace=True)

In [25]:
dfz.isnull().sum()

0

**When we check size column we can see that some of it name as bhk 
while some of it name as bedroom. eventhough 4 bhk and 4 bedroom is both are same 
, so to avoiding this problem i creating new column called bhk.
in that column i just give number only. ie if the number is 2 it means bhk
. for that i want to slice number from size column by using python split 
function**

In [26]:
dff['bhk'] = dfz.apply(lambda x: int(x.split(' ')[0]))

In [27]:
dff

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [28]:
dff['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [29]:
# now lets check is there any place where bedroom more than 20

In [30]:
dff[dff['bhk']>20]

,location,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,8000,27.0,230.0,27
4684,Munnekollal,2400,40.0,660.0,43


**while anlaysing this data you can see that there is 43 bedroom houses are there
but when we look to total sqft we can understand that.it is a mistake in sqft now we want to fix it**

In [31]:
dff['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

**in here we can see some of the values are not in float type ,
they are return in range format. this is the main part of EDA, ie 
most of the time dat are not given clear way, and we want to clean and fix it**

#for that i frist checking which all are the data are not float type

In [32]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [33]:
# printing the number that return true in float 
dff[dff['total_sqft'].apply(is_float)]

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [34]:
# printing the values that are not float in the column.
# for that code is same just put an indicator operation in front of column acces

dff[~dff['total_sqft'].apply(is_float)]

,location,total_sqft,bath,price,bhk
30,Yelahanka,2100 - 2850,4.0,186.000,4
56,Devanahalli,3010 - 3410,Whitefield,192.000,4
81,Hennur Road,2957 - 3450,Whitefield,224.500,4
122,Hebbal,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,1042 - 1105,2.0,54.005,2
...,...,...,...,...,...
12990,Talaghattapura,1804 - 2273,3.0,122.000,3
13059,Harlur,1200 - 1470,2.0,72.760,2
13240,Devanahalli,1020 - 1130,Whitefield,52.570,1
13265,Hoodi,1133 - 1384,2.0,59.135,2


In [35]:
# so the above are the range of value that not belong to float presnt in total sqr column

**but i checking it i see that whitfield value is enter too bath and for avoiding it
i using median of bath ie 2 replace with whitefield**

In [36]:
dff.loc[(dff['bath'])== 'Whitefield'] =2

In [37]:
dff['bath']

0        2.0
1        5.0
2        2.0
3        3.0
4        2.0
        ... 
13315    4.0
13316    5.0
13317    2.0
13318    4.0
13319    1.0
Name: bath, Length: 13320, dtype: object

In [38]:
dff[~dff['total_sqft'].apply(is_float)].head(30)

,location,total_sqft,bath,price,bhk
30,Yelahanka,2100 - 2850,4.0,186.000,4
122,Hebbal,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,1042 - 1105,2.0,54.005,2
165,Sarjapur,1145 - 1340,2.0,43.490,2
188,KR Puram,1015 - 1540,2.0,56.800,2
410,Kengeri,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,1195 - 1440,2.0,63.770,2
648,Arekere,4125Perch,9.0,265.000,9
661,Yelahanka,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,3090 - 5002,4.0,445.000,4


In [39]:
v=dff[~dff['total_sqft'].apply(is_float)].index

In [40]:
v


Int64Index([   30,   122,   137,   165,   188,   410,   549,   648,   661,
              672,
            ...
            12544, 12560, 12652, 12791, 12955, 12975, 12990, 13059, 13265,
            13299],
           dtype='int64', length=190)

In [41]:
dff.drop(v,axis=0,inplace=True)

**in the above step i drop the column that not belong in float type, i drop the 
value that come in range and in meter type**

In [42]:
#if u want to use it ,u can convert the range into float value by atking it average use this code to convert it
# def window(x):
#     tokens = x.split('-')
#     if len(tokens) == 2:
#         return (float(tokens[0])+float(tokens[1]))/2
#     try:
#         return float(x)
#     except:
#         return None 

**The above function is using to procees range into floating value
for example take range of 2100 - 2850 as x value. in first we split range besed on '-'
. then we store it j . and then we check j has 2 values, ie when we split range
we get two values in here its 2100 and 2850. if it is true then we take its average
and convert it into float value . if there single value ,ie not in type of range 
inside try we also coverting it in also float. but only demerit in here is
we cant convert 34.46Sq. Meter like numbers into float . for that we need 
another mathematic equtation for converting to squrt**

In [43]:
dff

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [44]:
dff['total_sqft'].dtype

dtype('O')

In [45]:
dff['total_sqft'] = dff['total_sqft'].astype(float)

In [46]:
dff['total_sqft'].dtype

dtype('float64')

In [47]:
dff['price'].dtype

dtype('float64')

In [48]:
#next i creating a newfeature called price_per_sqft
#it is avalue of price *lakh/total sqr feet
#i put lakh cuz we printing our price in form of lakh
#for doing this i want to change total sqrft dataype from 'o' to float ,and that i was written in above line

dff['price_per_sqft'] =dff['price']*100000/dff['total_sqft']

In [49]:
dff

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5,6689.834926
13316,Richards Town,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4689.0,4.0,488.00,4,10407.336319


**Handling the location column**

In [50]:
dff.location.nunique()

1299

In [51]:
# see in here we can see that total 1299 location has availble in data set, it is a huge number
# i am going to minimlize it based on the value count or datapoint available in location

#### Examine locations which is a categorical variable. We need to apply dimensionality reduction technique here to reduce number of locations

In [52]:

location_count = dff['location'].value_counts(ascending=False)
location_count

Whitefield                        518
Sarjapur  Road                    386
Electronic City                   302
Kanakpura Road                    260
Thanisandra                       222
                                 ... 
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
Kannur                              1
singapura paradise                  1
Abshot Layout                       1
Name: location, Length: 1299, dtype: int64

**in here you can see that there are more location in the column which has only one datapoint**
**now i am only taking the location which has value_count more than 10**

In [53]:
#lets check how many location in column has more than 10 count
len(location_count[location_count>10])

240

In [54]:
# there are 240 location in the column which has more than 10 valuecount or dataa point

In [55]:
#now lets check how manyy loaction has minimum 10 count

len(location_count>10)

1299

In [56]:
#there are 1299 rows in column are count less than 10, so in here i only taking rows which has location countb more than 10

#### Dimensionality Reduction

**Any location having less than 10 data points should be tagged as "other" location. This way number of categories can be reduced by huge amount. Later on when we do one hot encoding, it will help us with having fewer dummy columns**

In [57]:
#firstly i storing location count less than 10 into a variable
location_count_less_than_10 = location_count[location_count<=10]

In [58]:
location_count_less_than_10

Nagappa Reddy Layout              10
BTM 1st Stage                     10
ITPL                              10
Kalkere                           10
Ganga Nagar                       10
                                  ..
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
Kannur                             1
singapura paradise                 1
Abshot Layout                      1
Name: location, Length: 1059, dtype: int64

**next step i using lambda funcction, ie i accesing entire location column.if any row
of the column value is presnt  location_count_less_than_10 variable then i store that as other,
while rest of them that do not match with that variable store in the location**

In [59]:
dff.location = dff.location.apply(lambda x: 'other' if x in location_count_less_than_10 else x)
len(dff.location.unique())

241

**main advantage of doing this is, when i want to do categorical encoding
i just simply want to handle 241 values only**

In [60]:
dff['location'].value_counts()

other                 2891
Whitefield             518
Sarjapur  Road         386
Electronic City        302
Kanakpura Road         260
                      ... 
Pattandur Agrahara      11
Banjara Layout          11
Tindlu                  11
Kodigehalli             11
Nehru Nagar             11
Name: location, Length: 241, dtype: int64

#### Outlier Removal Using Business Logic

**As a data scientist when you have a conversation with your business manager (who has expertise in real estate), he will tell you that normally square ft per bedroom is 300 (i.e. 2 bhk apartment is minimum 600 sqft. If you have for example 400 sqft apartment with 2 bhk than that seems suspicious and can be removed as an outlier. We will remove such outliers by keeping our minimum thresold per bhk to be 300 sqft**

In [61]:
dff[dff.total_sqft/dff.bhk<300].head(10)

,location,total_sqft,bath,price,bhk,price_per_sqft
9,other,1020.0,6.0,370.0,6,36274.509804
45,HSR Layout,600.0,9.0,200.0,8,33333.333333
56,2,2.0,2,2.0,2,100000.000000
58,Murugeshpalya,1407.0,4.0,150.0,6,10660.980810
68,other,1350.0,7.0,85.0,8,6296.296296
70,other,500.0,3.0,100.0,3,20000.000000
78,Kaval Byrasandra,460.0,1.0,22.0,2,4782.608696
81,2,2.0,2,2.0,2,100000.000000
89,Rajaji Nagar,710.0,6.0,160.0,6,22535.211268
119,Hennur Road,276.0,3.0,23.0,2,8333.333333


**Check above data points. We have 6 bhk apartment with 1020 sqft. Another one is 8 bhk and total sqft is 600. These are clear data errors that can be removed safely**

In [62]:
#now iam going to remove this type unlogical column value
#you can use two method either collect all index store in variable and drop it
# else use indicator operation

In [63]:
dff.shape

(13130, 6)

In [64]:
# iam here using indicator operations
data=dff[~(dff.total_sqft/dff.bhk<300)]

In [65]:
data

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5,6689.834926
13316,other,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4689.0,4.0,488.00,4,10407.336319


In [66]:
#now we removed unlogical rows, u can see by looking shape diffrence

In [67]:
data.shape

(12313, 6)

In [68]:
#Next  we are going check whether outlayer is in price_per_sqft column , if so we handle it

In [69]:
data['price_per_sqft'].describe()

count     12313.000000
mean       6323.326489
std        4187.049729
min         267.829813
25%        4208.754209
50%        5300.000000
75%        6938.775510
max      176470.588235
Name: price_per_sqft, dtype: float64

**while taking describe we can see the min and maximum value in that column.**
**while looking that  we can see that minmum price_per_sqft is 267 in banglore, and
similarly the maximum price of its also seems to be very unlikely, so here i am going
to fix this issue by removing outlayer using standard deviation and mean**

**Outlier Removal Using Standard Deviation and Mean**